Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import pandas as pd, time
startNB = time.time()

# Ensemble Public Test

In [2]:
# LOAD XGBoost3 FIRST
sub = pd.read_parquet('../XGBoost3/sub_pub_1334_1.parquet')
for k in range(2,6):
    sub0 = pd.read_parquet('../XGBoost3/sub_pub_1334_%i.parquet'%k)
    sub.iloc[:,3:] += sub0.iloc[:,3:].values
sub.iloc[:,3:] /= 5.
sub.head()

,tweet_id,b_user_id,prediction,reply,retweet,retweet_comment,like
0,7647B4E9DAF4C1D8973397DC2A04F3E3,0000006C3074607050F1339DDCB890BB,0.0,0.031863,0.013925,0.003325,0.549908
1,408DB1803264B5FF55F73EC06BE9BD77,000013315386492275CCBF7AEF293EF0,0.0,0.002831,0.094954,0.002780,0.899267
2,2EE951379C47E8BF62EABB8FA027F753,00001569CB28972FC8173122D9DA162F,0.0,0.014427,0.064910,0.004155,0.341107
3,2135F24B05DAE3EF213F9CE80FDC6DAF,00001607209C5774DF9207A2AC0EED5F,0.0,0.001251,0.048451,0.001705,0.572710
4,09143FEDE9BD494A6EA9A7EE160565E3,0000177705514C315F2FC6DFA3872712,0.0,0.005884,0.031211,0.002068,0.205644


In [3]:
# LOAD XGBoost1 MODEL
sub0 = pd.read_csv('../XGBoost1/submissions/xgb-final-retweet-public-1.csv',header=None)
sub1 = pd.read_csv('../XGBoost1/submissions/xgb-final-reply-public-1.csv',header=None)
sub2 = pd.read_csv('../XGBoost1/submissions/xgb-final-like-public-1.csv',header=None)
sub3 = pd.read_csv('../XGBoost1/submissions/xgb-final-retweet_comment-public-1.csv',header=None)
sub3.head()

,0,1,2
0,7647B4E9DAF4C1D8973397DC2A04F3E3,0000006C3074607050F1339DDCB890BB,0.025758
1,408DB1803264B5FF55F73EC06BE9BD77,000013315386492275CCBF7AEF293EF0,0.002699
2,2EE951379C47E8BF62EABB8FA027F753,00001569CB28972FC8173122D9DA162F,0.000016
3,2135F24B05DAE3EF213F9CE80FDC6DAF,00001607209C5774DF9207A2AC0EED5F,0.006969
4,09143FEDE9BD494A6EA9A7EE160565E3,0000177705514C315F2FC6DFA3872712,0.000004


In [4]:
wt1 = [0.8, 0.7, 1.0, 0.7]
sub.retweet = (1-wt1[0]) * sub.retweet.values + wt1[0] * sub0.iloc[:,2].values
sub.reply = (1-wt1[1]) * sub.reply.values + wt1[1] * sub1.iloc[:,2].values
sub.like = (1-wt1[2]) * sub.like.values + wt1[2] * sub2.iloc[:,2].values
sub.retweet_comment = (1-wt1[3]) * sub.retweet_comment.values + wt1[3] * sub3.iloc[:,2].values

In [5]:
# LOAD XGBoost2 MODEL
sub0 = pd.read_csv('../XGBoost2/output/sub040/pub_retweet_x0.85.csv',header=None)
sub1 = pd.read_csv('../XGBoost2/output/sub040/pub_reply_x0.85.csv',header=None)
sub2 = pd.read_csv('../XGBoost2/output/sub040/pub_like_x0.85.csv',header=None)
sub3 = pd.read_csv('../XGBoost2/output/sub040/pub_retweet_comment_x0.85.csv',header=None)
sub3.head()

,0,1,2
0,7647B4E9DAF4C1D8973397DC2A04F3E3,0000006C3074607050F1339DDCB890BB,0.005483
1,408DB1803264B5FF55F73EC06BE9BD77,000013315386492275CCBF7AEF293EF0,0.000289
2,2EE951379C47E8BF62EABB8FA027F753,00001569CB28972FC8173122D9DA162F,0.000014
3,2135F24B05DAE3EF213F9CE80FDC6DAF,00001607209C5774DF9207A2AC0EED5F,0.002063
4,09143FEDE9BD494A6EA9A7EE160565E3,0000177705514C315F2FC6DFA3872712,0.000005


In [6]:
wt2 = [0.4, 0.5, 0.3, 0.6]
sub.retweet = (1-wt2[0]) * sub.retweet.values + wt2[0] * sub0.iloc[:,2].values
sub.reply = (1-wt2[1]) * sub.reply.values + wt2[1] * sub1.iloc[:,2].values
sub.like = (1-wt2[2]) * sub.like.values + wt2[2] * sub2.iloc[:,2].values
sub.retweet_comment = (1-wt2[3]) * sub.retweet_comment.values + wt2[3] * sub3.iloc[:,2].values

In [7]:
# WRITE FILE
for t in sub.columns[3:]:
    sub[t] = sub[t].astype('float32')
sub.to_parquet('submission_public.parquet',index=False)

In [ ]:
sub[['tweet_id','b_user_id','reply']].to_csv('public_sub_reply.csv',index=False,header=False)
sub[['tweet_id','b_user_id','retweet']].to_csv('public_sub_retweet.csv',index=False,header=False)
sub[['tweet_id','b_user_id','retweet_comment']].to_csv('public_sub_comment.csv',index=False,header=False)
sub[['tweet_id','b_user_id','like']].to_csv('public_sub_like.csv',index=False,header=False)

# Ensemble Private Test

In [8]:
# LOAD XGBoost3 FIRST
sub = pd.read_parquet('../XGBoost3/sub_priv_1334_1.parquet')
for k in range(2,6):
    sub0 = pd.read_parquet('../XGBoost3/sub_priv_1334_%i.parquet'%k)
    sub.iloc[:,3:] += sub0.iloc[:,3:].values
sub.iloc[:,3:] /= 5.
sub.head()

,tweet_id,b_user_id,prediction,reply,retweet,retweet_comment,like
0,04746004AA1F5498834CE7A4C6343D1A,00000776B07587ECA9717BFC301F2D6E,0.0,0.243955,0.006709,0.002323,0.917481
1,B5C4CBE185831F3E5A58A4D81118D4C7,00000776B07587ECA9717BFC301F2D6E,0.0,0.068159,0.197304,0.009618,0.913009
2,6B6836351BFAA6D1CC1EB0386BCB8C6A,00000B85AAF7DE172876FD96718C4469,0.0,0.007112,0.764966,0.091289,0.709978
3,0DCF558E40500F22F84F98C4E7C38EDC,00000E0C9B364891CDE89ECFC54771DE,0.0,0.004229,0.081783,0.005452,0.415365
4,F13AA57F12DD6107D9D8544A27BDE9EC,0000109A57AFA64758EE4AAE2A01BFC7,0.0,0.007811,0.015649,0.001693,0.108730


In [9]:
# LOAD XGBoost1 MODEL
sub0 = pd.read_csv('../XGBoost1/submissions/xgb-retweet-private-1.csv',header=None)
sub1 = pd.read_csv('../XGBoost1/submissions/xgb-reply-private-1.csv',header=None)
sub2 = pd.read_csv('../XGBoost1/submissions/xgb-like-private-1.csv',header=None)
sub3 = pd.read_csv('../XGBoost1/submissions/xgb-retweet_comment-private-1.csv',header=None)
sub3.head()

,0,1,2
0,04746004AA1F5498834CE7A4C6343D1A,00000776B07587ECA9717BFC301F2D6E,0.001207
1,B5C4CBE185831F3E5A58A4D81118D4C7,00000776B07587ECA9717BFC301F2D6E,0.004615
2,6B6836351BFAA6D1CC1EB0386BCB8C6A,00000B85AAF7DE172876FD96718C4469,0.138609
3,0DCF558E40500F22F84F98C4E7C38EDC,00000E0C9B364891CDE89ECFC54771DE,0.006942
4,F13AA57F12DD6107D9D8544A27BDE9EC,0000109A57AFA64758EE4AAE2A01BFC7,0.000853


In [10]:
sub.retweet = (1-wt1[0]) * sub.retweet.values + wt1[0] * sub0.iloc[:,2].values
sub.reply = (1-wt1[1]) * sub.reply.values + wt1[1] * sub1.iloc[:,2].values
sub.like = (1-wt1[2]) * sub.like.values + wt1[2] * sub2.iloc[:,2].values
sub.retweet_comment = (1-wt1[3]) * sub.retweet_comment.values + wt1[3] * sub3.iloc[:,2].values

In [11]:
# LOAD XGBoost2 MODEL
sub0 = pd.read_csv('../XGBoost2/output/sub040/pvt_retweet_x0.85.csv',header=None)
sub1 = pd.read_csv('../XGBoost2/output/sub040/pvt_reply_x0.85.csv',header=None)
sub2 = pd.read_csv('../XGBoost2/output/sub040/pvt_like_x0.85.csv',header=None)
sub3 = pd.read_csv('../XGBoost2/output/sub040/pvt_retweet_comment_x0.85.csv',header=None)
sub3.head()

,0,1,2
0,04746004AA1F5498834CE7A4C6343D1A,00000776B07587ECA9717BFC301F2D6E,0.000966
1,B5C4CBE185831F3E5A58A4D81118D4C7,00000776B07587ECA9717BFC301F2D6E,0.005360
2,6B6836351BFAA6D1CC1EB0386BCB8C6A,00000B85AAF7DE172876FD96718C4469,0.091135
3,0DCF558E40500F22F84F98C4E7C38EDC,00000E0C9B364891CDE89ECFC54771DE,0.005561
4,F13AA57F12DD6107D9D8544A27BDE9EC,0000109A57AFA64758EE4AAE2A01BFC7,0.001984


In [12]:
sub.retweet = (1-wt2[0]) * sub.retweet.values + wt2[0] * sub0.iloc[:,2].values
sub.reply = (1-wt2[1]) * sub.reply.values + wt2[1] * sub1.iloc[:,2].values
sub.like = (1-wt2[2]) * sub.like.values + wt2[2] * sub2.iloc[:,2].values
sub.retweet_comment = (1-wt2[3]) * sub.retweet_comment.values + wt2[3] * sub3.iloc[:,2].values

In [13]:
# WRITE FILE
for t in sub.columns[3:]:
    sub[t] = sub[t].astype('float32')
sub.to_parquet('submission_private.parquet',index=False)

In [ ]:
sub[['tweet_id','b_user_id','reply']].to_csv('private_sub_reply.csv',index=False,header=False)
sub[['tweet_id','b_user_id','retweet']].to_csv('private_sub_retweet.csv',index=False,header=False)
sub[['tweet_id','b_user_id','retweet_comment']].to_csv('private_sub_comment.csv',index=False,header=False)
sub[['tweet_id','b_user_id','like']].to_csv('private_sub_like.csv',index=False,header=False)

In [14]:
print('Elapsed Time is %f minutes'%((time.time()-startNB)/60))

Elapsed Time is 5.787362 minutes
